# `PyBird` likelihoods

In this notebook we calculate likelihoods with `PyBird` for samples from a MCMC chain generated with the `EFTEMU`. These likelihoods will then be used to calculate importance weights.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matryoshka.eft_funcs as MatEFT
from scipy.interpolate import interp1d
from scipy.stats import norm
from classy import Class
import pybird
from tqdm import tqdm

In [2]:
path_to_repo = "/Users/jamie/Desktop/GitHubProjects/matryoshka_II_paper/"

We start by loading the mock multipoles at $z=0.61$ and covaraince with $V_s^{1/3}=5000\ \mathrm{Mpc}\ h^{-1}$.

In [3]:
P0 = np.load(path_to_repo+"data/P18/z0.61/poles/P0_P18--z-0.61_optiresum-False.npy")[1]
P2 = np.load(path_to_repo+"data/P18/z0.61/poles/P2_P18--z-0.61_optiresum-False.npy")[1]
klin = np.load(path_to_repo+"data/P18/z0.61/poles/P2_P18--z-0.61_optiresum-False.npy")[0]
cov = np.load(path_to_repo+"data/P18/z0.61/covs/cov_P18--z-0.61_Vs-5000.npy")
icov = np.linalg.inv(cov)

We now define the truths. These are only used to fix the relevent parameters.

In [5]:
true_cosmo = np.array([0.11933, 0.02242, 0.6766, 3.047, 0.9665])
fb = true_cosmo[1]/(true_cosmo[0]+true_cosmo[1])
ng = 3e-4
bs_CMASS = np.array([2.22, 1.2, 0.1, 0.0, 0.4, -7.7, 0., 0., 0., -3.7])

Next we define the likelihood. This has the same, Gaussia, form as that used when running the MCMC with the `EFTEMU`. See `mcmc_z0.61_v3700_EFTEMU.ipynb`. If should be noted that the function is now designed to work with single parameter sets rather than a batch as with the `EFTEMU`.

In [ ]:
def log_like(theta, ks, obs, icov, ng):
    
    res = np.concatenate(power_pred(theta, ng, ks))-obs

    return -0.5*np.dot(np.dot(res,icov),res.T)

In [ ]:
def power_pred(theta, ng, ks):
    cosmo = theta[:5] # Oc, Ob, h, As, ns
    bias = theta[5:12] # b1, c2, b3, c4, cct, cr1, cr2
    stoch = theta[12:] # ce1, cmono, cquad
        
    c2 = np.copy(bias[1])
    c4 = np.copy(bias[3])
    
    bias[1] = (c2+c4)/np.sqrt(2)
    bias[3] = (c2-c4)/np.sqrt(2)
    
    M.set({'ln10^{10}A_s': cosmo[3],
           'n_s': cosmo[4],
           'h': cosmo[2],
           'omega_b': cosmo[1],
           'omega_cdm': cosmo[0],
          })
    
    # Calculate the linear power spectrum.
    M.compute()
        
    # Convert to (Mpc/h)^3.
    Pk = [M.pk(ki*M.h(), 0.61)*M.h()**3 for ki in kk]
    
    f = M.scale_independent_growth_factor_f(0.61)
    
    bird = pybird.Bird(kk, Pk, f, z=0.61, which='all', co=common)

    # Calculate the desired functions.
    nonlinear.PsCf(bird)
    bird.setPsCfl()
    resum.Ps(bird)
    
    # Compute multipoles from Pn.
    P0_pred = MatEFT.multipole([bird.P11l[0][:,:39], bird.Ploopl[0][:,:39], bird.Pctl[0][:,:39]], 
                               bias, f, stochastic=stoch, ng=ng, multipole=0,
                               kbins=pybird.kbird[:39])
 
    P2_pred = MatEFT.multipole([bird.P11l[1][:,:39], bird.Ploopl[1][:,:39], bird.Pctl[1][:,:39]], 
                               bias, f, stochastic=stoch, ng=ng, multipole=2,
                               kbins=pybird.kbird[:39])
    
    return interp1d(pybird.kbird[:39], P0_pred)(ks), interp1d(pybird.kbird[:39], P2_pred)(ks)

We also define the function `fix_params()` that allows us to easily fix parameters.

In [6]:
def fix_params(theta, fix_val, fb):
    var_id = np.array([0,2,3,5,6,7,9,10,12,14])
    fix_id = np.array([4,8,11,13])
    fix_theta = np.zeros((15, ))
    fix_theta[var_id] = theta
    fix_theta[fix_id] = fix_val
    fix_theta[1] = -fb*theta[0]/(fb-1)
    return fix_theta

`CLASS` and `PyBird` set-up.

In [7]:
M = Class()
M.set({'output': 'mPk',
       'P_k_max_1/Mpc': 1.0,
       'z_max_pk': 0.61})
common = pybird.Common(optiresum=False)
nonlinear = pybird.NonLinear(load=True, save=True, co=common)
resum = pybird.Resum(co=common)
kk = np.logspace(-5, 0, 200)

We next load the MCMC chain generated with the `EFTEMU`. **It should be noted that this chain is generated with $V_s^{1/3}=3700\ \mathrm{Mpc}\ h^{-1}$**. Using this smaller volume inflates the posterior and allows us to mitigate any risk of loosing tails of the `PyBird` posterior.

In [8]:
emu_chain = np.load(path_to_repo+"results/chain--EFTEMU_z-0.61_V-3700_kmin-def_kmax-def_5.npy")

Loop over the samples in the chain and calculate the log-likelihood.

In [9]:
likes = np.zeros((emu_chain.shape[0]))
for i, theta in tqdm(enumerate(emu_chain)):
    likes[i] = log_like(fix_params(theta, np.array([true_cosmo[-1],0.,0.,0.]), fb),
                                   klin, np.concatenate([P0,P2]), icov, ng)

1960it [1:13:57,  2.26s/it]


In [10]:
np.save(path_to_repo+"results/pybird_likes--EFTEMU_z-0.61_Vc-3700_Vi-5000_kmin-def_kmax-def_5.npy", likes)